In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/kaggle_jobs_raw.csv")

df.head()


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,EX,FT,Head of Data,348516,USD,348516,US,0,US,M
1,2025,EX,FT,Head of Data,232344,USD,232344,US,0,US,M
2,2025,SE,FT,Data Scientist,145400,USD,145400,US,0,US,M
3,2025,SE,FT,Data Scientist,81600,USD,81600,US,0,US,M
4,2025,MI,FT,Engineer,160000,USD,160000,US,100,US,M


In [2]:
df.shape

(151445, 11)

In [3]:
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151445 entries, 0 to 151444
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   work_year           151445 non-null  int64 
 1   experience_level    151445 non-null  object
 2   employment_type     151445 non-null  object
 3   job_title           151445 non-null  object
 4   salary              151445 non-null  int64 
 5   salary_currency     151445 non-null  object
 6   salary_in_usd       151445 non-null  int64 
 7   employee_residence  151445 non-null  object
 8   remote_ratio        151445 non-null  int64 
 9   company_location    151445 non-null  object
 10  company_size        151445 non-null  object
dtypes: int64(4), object(7)
memory usage: 12.7+ MB


In [6]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [8]:
print(df.duplicated().sum())


79532


In [9]:
df = df.drop_duplicates()
df.shape

(71913, 11)

In [10]:
def clean_job_title(title):
    title = str(title).lower()
    
    if "data analyst" in title:
        return "Data Analyst"
    elif "data scientist" in title:
        return "Data Scientist"
    elif "machine learning" in title or "ml engineer" in title:
        return "ML Engineer"
    elif "ai engineer" in title or "artificial intelligence" in title:
        return "AI Engineer"
    elif "business analyst" in title:
        return "Business Analyst"
    elif "data engineer" in title:
        return "Data Engineer"
    else:
        return "Other"

df["job_title_clean"] = df["job_title"].apply(clean_job_title)


In [11]:
df["job_title_clean"].value_counts()


job_title_clean
Other               45313
Data Scientist       7216
Data Engineer        7112
Data Analyst         6490
ML Engineer          4221
AI Engineer          1211
Business Analyst      350
Name: count, dtype: int64

In [12]:
df["salary_usd"] = df["salary_in_usd"]


In [15]:
df = df[df["salary_usd"].notna()]
df = df[df["salary_usd"] > 0]



In [16]:
experience_map = {
    "entry-level": 1,
    "mid-level": 2,
    "senior-level": 3,
    "executive-level": 4
}

df["experience_rank"] = df["experience_level"].map(experience_map)


In [17]:
df["company_country"] = df["company_location"].astype(str).str.split(",").str[-1].str.strip()


In [18]:
df["requires_python"] = df["job_title"].str.lower().str.contains("python", na=False).astype(int)
df["requires_sql"] = df["job_title"].str.lower().str.contains("sql", na=False).astype(int)
df["requires_ml"] = df["job_title"].str.lower().str.contains("machine learning|ml", na=False).astype(int)
df["requires_ai"] = df["job_title"].str.lower().str.contains("ai|artificial intelligence", na=False).astype(int)


In [19]:
df.to_csv("../data/processed/jobs_clean.csv", index=False)
